In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow warnings

zip_path ='/content/drive/MyDrive/rock-paper-scissors'  # Path to the main extracted folder

def load_dataset(zip_path):
    """
    Load and preprocess the dataset.
    Args:
        zip_path (str): Path to the extracted folder.
    Returns:
        X (np.array): Preprocessed images.
        y (np.array): Corresponding labels.
    """
    classes = {'rock': 0, 'paper': 1, 'scissors': 2}  # Mapping gestures to labels
    X, y = [], []

    # Load all images and labels from the 'train' subset
    subset_path = os.path.join(zip_path, 'train')

    for label, value in classes.items():
        class_path = os.path.join(subset_path, label)

        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale
            img = cv2.resize(img, (64, 64))  # Resize to 64x64
            X.append(img)
            y.append(value)

    X = np.array(X).reshape(-1, 64, 64, 1) / 255.0  # Normalize
    y = np.array(y)
    return X, y

In [ ]:
X, y = load_dataset(zip_path)
print(f"Dataset shape: {X.shape}, Labels shape: {y.shape}")


Dataset shape: (8572, 64, 64, 1), Labels shape: (8572,)


In [ ]:
# Split the data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)  # 60% train, 40% temp
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 20% val, 20% test
print(f"Training data shape: {X_train.shape}, Validation data shape: {X_val.shape}, Testing data shape: {X_test.shape}")

Training data shape: (5143, 64, 64, 1), Validation data shape: (1714, 64, 64, 1), Testing data shape: (1715, 64, 64, 1)


In [ ]:
def build_model():
    """
    Build a CNN model for hand gesture recognition.
    """
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(3, activation='softmax')  # 3 classes: rock, paper, scissors
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 62, 62, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       1,605,760 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,624,963 (6.20 MB)

 Trainable params: 1,624,963 (6.20 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32)

Epoch 1/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 31s 183ms/step - accuracy: 0.4898 - loss: 0.9656 - val_accuracy: 0.7369 - val_loss: 0.6125
Epoch 2/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 41s 182ms/step - accuracy: 0.7823 - loss: 0.5242 - val_accuracy: 0.8594 - val_loss: 0.3900
Epoch 3/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 40s 178ms/step - accuracy: 0.8812 - loss: 0.3161 - val_accuracy: 0.8600 - val_loss: 0.3539
Epoch 4/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 41s 176ms/step - accuracy: 0.9212 - loss: 0.2159 - val_accuracy: 0.8961 - val_loss: 0.2896
Epoch 5/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 40s 169ms/step - accuracy: 0.9580 - loss: 0.1300 - val_accuracy: 0.9142 - val_loss: 0.2533
Epoch 6/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 42s 175ms/step - accuracy: 0.9746 - loss: 0.0854 - val_accuracy: 0.9195 - val_loss: 0.2594
Epoch 7/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 41s 179ms/step - accuracy: 0.9813 - loss: 0.0626 - val_accuracy: 0.9236 - val_loss: 0.2512
Epoch 8/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 41s 179ms/step - accuracy: 0.9908 - loss: 0

In [ ]:
# Save the model
model.save(r"C:\Users\DELL\Desktop\my_model.keras")

NameError: name 'model' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model.save('/content/drive/MyDrive/my_model.keras')


In [ ]:
!ls /content/drive/My\ Drive



'Colab Notebooks'      my_model.keras	     Rock-Paper-Scissors
 gesture_model.keras   rock-paper-scissors   test_image.jpg
